In [ ]:
#############  deploy model from the model registry #######################

model_package_name = 'diabates-main-structured'
model_version = 4  # Your model version
#model_package_arn = f"arn:aws:sagemaker:{region}:590183717898:model-package/{model_package_name}/{model_version}"

import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define your model package ARN (from the Model Registry)
model_package_arn = f"arn:aws:sagemaker:{region}:590183717898:model-package/{model_package_name}/{model_version}"

# Create the model from the registered model in the model registry using Model Package ARN
model_name = 'zoheb-test-inference-1'  # Give your model a unique name
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=sagemaker_role,
    Containers=[  # Use Containers when specifying a model package ARN
        {
            'ModelPackageName': model_package_arn
        }
    ]
)

print("Model creation response:", create_model_response)

# Create the endpoint configuration
endpoint_config_name = 'diabates-endpoint-config-3'  # Unique name for the endpoint config
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InitialVariantWeight': 1,
            'InstanceType': 'ml.m5.large',  # Choose your instance type
            'InitialInstanceCount': 1
        }
    ]
)

print("Endpoint configuration created:", endpoint_config_response['EndpointConfigArn'])

# Create the endpoint
endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=model_name,  # Endpoint name should match the model name
    EndpointConfigName=endpoint_config_name  # Use the config you just created
)

print("Endpoint creation initiated:", endpoint_response['EndpointArn'])


In [ ]:
###### to retrieve the latest model which is registered in model registry##################

import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define your model package group
model_package_group_name = 'diabates-main-structured'

def get_latest_approved_model():
    # List model packages in the specified group, sorted by creation time (latest first)
    response = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1,  # Only get the latest model
        ModelApprovalStatus='Approved'  # Optional: Filter by approved status
    )

    # Check if there are any approved models
    if len(response['ModelPackageSummaryList']) > 0:
        # Retrieve the ARN of the latest model
        latest_model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
        print(f"Latest approved model ARN: {latest_model_package_arn}")
        return latest_model_package_arn
    else:
        print("No approved models found.")
        return None

# Get the latest approved model ARN
latest_model_arn = get_latest_approved_model()

if latest_model_arn:
    # Now you can use this ARN to deploy the model to an endpoint
    # For example, update an endpoint or create a new endpoint
    print(f"Deploying model: {latest_model_arn}")
else:
    print("Model not found or approval is missing.")